In [1]:
import gc
import os
import random
import time
import torch
import datetime
import numpy as np
import pandas as pd
import polars as pl
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.regression import R2Score

In [2]:
DATA_PATH = "/kaggle/input/"
BATCH_SIZE = 13000
MIN_STD = 1e-8
SCHEDULER_PATIENCE = 3
SCHEDULER_FACTOR = 10**(-0.5)
EPOCHS = 70
PATIENCE = 6
PRINT_FREQ = 50

In [3]:
def format_time(elapsed):
    """Take a time in seconds and return a string hh:mm:ss."""
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [4]:
def seed_everything(seed_val=1325):
    """Seed everything."""
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

In [5]:
def columns_with_low_variance(X):
    variances = X.select([pl.col(column).var() for column in X.columns])
#     variances
    low_variance_cols = [col for col, var in zip(X.columns, variances.row(0)) if var < 1e-6]
    print(len(low_variance_cols))
    return low_variance_cols
# Define normalization functions
def normalize_min_max(x, mean, min_val, max_val):
    return (x - mean) / (max_val - min_val)

def normalize_div_max(x, max_val):
    return x / max_val

def normalize_exp(x, lambda_val):
    return 1 - np.exp(-lambda_val * x)

In [6]:
# df_train = pl.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/train.csv", n_rows=100)
def normalize(df_train,FEAT_COLS):
    FEAT_COLS = df_train.columns[1:557]
    TARGET_COLS = df_train.columns[557:]
    state_t_cols = []#(x-mean)/(max-min)
    state_q0001_cols = []# no norm
    state_q0002_cols = []#1-exp()
    state_q0003_cols = []#1-exp()
    state_u_cols = []#(x-mean)/(max-min)
    state_v_cols = []#(x-mean)/(max-min)
    pbuf_ozone_cols = []#(x-mean)/(max-min)
    pbuf_CH4_cols = []#(x-mean)/(max-min)
    pbuf_N2O_cols = []#(x-mean)/(max-min)
    centered_min_max_cols = ['state_ps','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_SNOWHLAND']
    min_max_scaling_cols = ['pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX']
    expo_scaling_cols = []

    for col in FEAT_COLS:
        if 'state_t'in col:
            state_t_cols.append(col)
            centered_min_max_cols.append(col)
        elif 'state_q0001' in col:
            state_q0001_cols.append(col)
        elif 'state_q0002' in col:
            state_q0002_cols.append(col)
            expo_scaling_cols.append(col)
        elif 'state_q0003' in col:
            state_q0003_cols.append(col)
            expo_scaling_cols.append(col)
        elif 'state_u' in col:
            state_u_cols.append(col)
            centered_min_max_cols.append(col)
        elif 'state_v' in col:
            state_v_cols.append(col)
            centered_min_max_cols.append(col)
        elif 'pbuf_ozone' in col:
            pbuf_ozone_cols.append(col)
            centered_min_max_cols.append(col)
        elif 'pbuf_CH4' in col:
            pbuf_CH4_cols.append(col)
            centered_min_max_cols.append(col)
        elif 'pbuf_N2O' in col:
            pbuf_N2O_cols.append(col)
            centered_min_max_cols.append(col)
    lis_vector_features = [state_t_cols,state_q0001_cols,state_q0002_cols,state_q0003_cols,state_u_cols,state_v_cols,pbuf_ozone_cols,pbuf_CH4_cols,pbuf_N2O_cols]
    
    for col in FEAT_COLS:
        if col in centered_min_max_cols:
            mean = df_train[col].mean()
            min_val = df_train[col].min()
            max_val = df_train[col].max()
            if not (min_val > 0 and max_val < 1):
                df_train = df_train.with_columns(normalize_min_max(pl.col(col), mean, min_val, max_val).alias(col))
        if col in min_max_scaling_cols:
            min_val = df_train[col].min()
            max_val = df_train[col].max()
            if not (min_val > 0 and max_val < 1):
                df_train = df_train.with_columns(normalize_div_max(pl.col(col), max_val).alias(col))
        if col in expo_scaling_cols:
            try:
                lambda_val = 1/df_train[col].filter(df_train[col]>1e-7).mean()
            except Exception as e:
                lambda_val = 1
            df_train = df_train.with_columns(normalize_exp(pl.col(col), lambda_val).alias(col))
    return df_train,lis_vector_features

In [7]:
ts = time.time()

weights = pd.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/sample_submission.csv", nrows=1)
del weights['sample_id']
weights = weights.T
weights = weights.to_dict()[0]
batch_size = 2_500_000
batch_number = 2  # Change this to 2, 3, etc., to load subsequent batches
skip_rows = batch_size * batch_number

df_header = pl.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/train.csv", n_rows=1)
column_names = df_header.columns
# print(column_names)

df_train = pl.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/train.csv", n_rows=2_500_000, skip_rows_after_header=skip_rows)#2500000

# for target in weights:
#     df_train = df_train.with_columns(pl.col(target).mul(weights[target]))

print("Time to read dataset:", format_time(time.time()-ts), flush=True)

FEAT_COLS = df_train.columns[1:557]
TARGET_COLS = df_train.columns[557:]
df_train,list_vector_features = normalize(df_train,FEAT_COLS)
for col in FEAT_COLS:
    df_train = df_train.with_columns(pl.col(col).cast(pl.Float32))

for col in TARGET_COLS:
    df_train = df_train.with_columns(pl.col(col).cast(pl.Float32))

 
    
x_train = df_train.select(FEAT_COLS).to_numpy()
# low_variance_cols = columns_with_low_variance(x_train).to_numpy()
# x_train = x_train.drop(low_variance_cols).to_numpy()
y_train = df_train.select(TARGET_COLS).to_numpy()

# del df_train
# gc.collect()

# Normalization of training data
# mx = x_train.mean(axis=0)
# sx = np.maximum(x_train.std(axis=0), MIN_STD)
# # print('mean: ', mx, ' variance: ' sx)
# x_train = (x_train - mx.reshape(1,-1)) / sx.reshape(1,-1)

my = y_train.mean(axis=0)
sy = np.maximum(np.sqrt((y_train*y_train).mean(axis=0)), MIN_STD)
y_train = (y_train - my.reshape(1,-1)) / sy.reshape(1,-1)

print("Time after processing data:", format_time(time.time()-ts), flush=True)

Time to read dataset: 0:41:41
Time after processing data: 0:42:10


# checking data 

In [8]:
df_train.columns

['sample_id',
 'state_t_0',
 'state_t_1',
 'state_t_2',
 'state_t_3',
 'state_t_4',
 'state_t_5',
 'state_t_6',
 'state_t_7',
 'state_t_8',
 'state_t_9',
 'state_t_10',
 'state_t_11',
 'state_t_12',
 'state_t_13',
 'state_t_14',
 'state_t_15',
 'state_t_16',
 'state_t_17',
 'state_t_18',
 'state_t_19',
 'state_t_20',
 'state_t_21',
 'state_t_22',
 'state_t_23',
 'state_t_24',
 'state_t_25',
 'state_t_26',
 'state_t_27',
 'state_t_28',
 'state_t_29',
 'state_t_30',
 'state_t_31',
 'state_t_32',
 'state_t_33',
 'state_t_34',
 'state_t_35',
 'state_t_36',
 'state_t_37',
 'state_t_38',
 'state_t_39',
 'state_t_40',
 'state_t_41',
 'state_t_42',
 'state_t_43',
 'state_t_44',
 'state_t_45',
 'state_t_46',
 'state_t_47',
 'state_t_48',
 'state_t_49',
 'state_t_50',
 'state_t_51',
 'state_t_52',
 'state_t_53',
 'state_t_54',
 'state_t_55',
 'state_t_56',
 'state_t_57',
 'state_t_58',
 'state_t_59',
 'state_q0001_0',
 'state_q0001_1',
 'state_q0001_2',
 'state_q0001_3',
 'state_q0001_4',
 'stat

In [11]:
del df_train
gc.collect()

90

In [10]:
x_train[0]

array([-1.32552311e-02, -4.14982811e-03, -5.66318855e-02, -1.69730056e-02,
        2.06712186e-02,  6.52998984e-02,  9.44697931e-02,  9.14321467e-02,
        9.66449082e-02,  1.08857006e-01,  1.03046291e-01,  9.82882157e-02,
        9.78412926e-02,  8.99169892e-02,  8.54050815e-02,  9.81084853e-02,
        1.21981725e-01,  1.10161580e-01,  5.51250204e-02,  3.89608066e-03,
       -7.47085288e-02, -7.01557696e-02, -7.57217482e-02, -3.05965412e-02,
        1.86102726e-02,  6.53694794e-02,  9.83350426e-02,  1.23026609e-01,
        1.44744471e-01,  1.58854455e-01,  1.68501899e-01,  1.70721933e-01,
        1.71828687e-01,  1.70129851e-01,  1.64476871e-01,  1.61277086e-01,
        1.56901315e-01,  1.50220811e-01,  1.50779739e-01,  1.49541050e-01,
        1.47583395e-01,  1.40399694e-01,  1.33488148e-01,  1.29677147e-01,
        1.25084072e-01,  1.24075353e-01,  1.20844051e-01,  1.18244655e-01,
        1.14833243e-01,  1.12382591e-01,  1.09002866e-01,  1.06163077e-01,
        1.04597658e-01,  

In [31]:
del x_train 
del y_train
gc.collect()

0

# NN training

In [8]:
seed_everything()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
class ClimateDataset(Dataset):
    def __init__(self, x, y):
        """
        Initialize with NumPy arrays.
        """
        assert x.shape[0] == y.shape[0], "Features and labels must have the same number of samples"
        self.x = x
        self.y = y

    def __len__(self):
        """
        Total number of samples.
        """
        return self.x.shape[0]

    def __getitem__(self, index):
        """
        Generate one sample of data.
        """
        # Convert the data to tensors when requested
        return torch.from_numpy(self.x[index]).float().to(device), torch.from_numpy(self.y[index]).float().to(device)

In [10]:
dataset = ClimateDataset(x_train, y_train)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFNN, self).__init__()
        
        layers = []
        previous_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(previous_size, hidden_size))
#             layers.append(nn.LayerNorm(hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ELU(inplace=True))
#             layers.append(nn.LeakyReLU(inplace=True))
            layers.append(nn.Dropout(p=0.3))
            previous_size = hidden_size
        
#         layers.append(nn.Linear(previous_size, output_size))
        
        self.layers = nn.Sequential(*layers)
        self.output = nn.ModuleList([nn.Linear(previous_size, 1) for _ in range(output_size)])
        
    def forward(self, x):
        x =  self.layers(x)
        outputs = [output_layer(x) for output_layer in self.output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [ ]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFNN, self).__init__()
        
        layers = []
        previous_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(previous_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ELU(inplace=True))
            layers.append(nn.Dropout(p=0.5))
            previous_size = hidden_size
        
        self.layers = nn.Sequential(*layers)
        self.output_layers = nn.ModuleList([nn.Sequential(
            nn.Linear(previous_size, 128),
            nn.BatchNorm1d(128),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(64, 1)
        ) for _ in range(output_size)])
        
    def forward(self, x):
        x = self.layers(x)
        outputs = [output_layer(x) for output_layer in self.output_layers]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [ ]:
class FFNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFNN, self).__init__()
        
        layers = []
        previous_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(previous_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ELU(inplace=True))
            layers.append(nn.Dropout(p=0.3))
            previous_size = hidden_size
        
        self.layers = nn.Sequential(*layers)
        
        # Adding more layers to the final regression head
        self.output_layers = nn.ModuleList([nn.Sequential(
            nn.Linear(previous_size, 128),
            nn.BatchNorm1d(128),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(64, 1)
        ) for _ in range(output_size)])
        
    def forward(self, x):
        x = self.layers(x)
        outputs = [output_layer(x) for output_layer in self.output_layers]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [ ]:
input_size = x_train.shape[1]
output_size = y_train.shape[1]
# hidden_size = 100 #input_size + output_size
# model = FFNN(input_size, [hidden_size, int(0.5*hidden_size), int(0.5*hidden_size), hidden_size], output_size).to(device)
hidden_sizes = [512, 256, 128, 64, 128, 256, 512]#[256, 128, 64, 128, 256]
model = FFNN(input_size, hidden_sizes, output_size).to(device)
criterion = nn.HuberLoss()#nn.MSELoss() #nn.HuberLoss()#
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)

print("Time after all preparations:", format_time(time.time()-ts), flush=True)

In [ ]:
model

In [ ]:
best_val_loss = float('inf')
best_r2_score = -1000
best_model_state = None
patience_count = 0
r2score = R2Score(num_outputs=len(TARGET_COLS)).to(device)
EPOCHS = 80
for epoch in range(EPOCHS):
    print("")
    model.train()
    total_loss = 0
    steps = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        steps += 1

        if (batch_idx + 1) % PRINT_FREQ == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            elapsed_time = format_time(time.time() - ts)
            print(f'  Epoch: {epoch+1}',\
                  f'  Batch: {batch_idx + 1}/{len(train_loader)}',\
                  f'  Train Loss: {total_loss / steps:.4f}',\
                  f'  LR: {current_lr:.1e}',\
                  f'  Time: {elapsed_time}', flush=True)
            total_loss = 0
            steps = 0
    

    model.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)
    r2=0
    r2_broken = []
    r2_broken_names = []
    for i in range(368):
        r2_i = r2score(all_outputs[:, i], y_true[:, i])
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
#     r2 = r2score(all_outputs,y_true)
    r2 /= 368

    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
    # print(r2_broken)
    # print(r2_broken_names, flush=True)
   
    scheduler.step(avg_val_loss)

#     if avg_val_loss < best_val_loss:
    if r2 > best_r2_score:
        best_model_state = model.state_dict()
        best_r2_score = r2
        print("R2 score improved, saving new best model.")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_count = 0
        print("Validation loss decreased, resetting patience counter.")
    else:
        patience_count += 1
        print(f"No improvement in validation loss for {patience_count} epochs.")
        
    if patience_count >= PATIENCE:
        print("Stopping early due to no improvement in validation loss.")
        break

# del x_train, y_train
# gc.collect()

In [ ]:
num_epochs = 80
best_val_loss = float('inf')
best_r2_score = -1000
best_model_state = None
patience_count = 0
PATCIENCE = 10
PRINT_FREQ = 50

model = CustomModel_Unet_MLP(input_dim, output_dim, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=num_epochs)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    print("")
    model.train()
    total_loss = 0
    steps = 0
    ts = time.time()

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        steps += 1

        if (batch_idx + 1) % PRINT_FREQ == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            elapsed_time = time.time() - ts
            print(f'  Epoch: {epoch+1}',\
                  f'  Batch: {batch_idx + 1}/{len(train_loader)}',\
                  f'  Train Loss: {total_loss / steps:.4f}',\
                  f'  LR: {current_lr:.1e}',\
                  f'  Time: {elapsed_time:.2f}s', flush=True)
            total_loss = 0
            steps = 0

    model.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)

    r2 = 0
    r2_broken = []
    r2_broken_names = []
    for i in range(len(TARGET_COLS)):
        r2_i = r2_score(y_true[:, i].cpu(), all_outputs[:, i].cpu())
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
    r2 /= len(TARGET_COLS)

    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')

    scheduler.step(avg_val_loss)

    if r2 > best_r2_score:
        best_model_state = model.state_dict()
        best_r2_score = r2
        print("R2 score improved, saving new best model.")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_count = 0
        print("Validation loss decreased, resetting patience counter.")
    else:
        patience_count += 1
        print(f"No improvement in validation loss for {patience_count} epochs.")
        
    if patience_count >= PATIENCE:
        print("Stopping early due to no improvement in validation loss.")
        break

model.load_state_dict(best_model_state)
print("Training complete. Best R2 score: {:.4f}".format(best_r2_score))

# NN 2

In [11]:
import torch
from torch import nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)  # "same" padding in TF translates to padding=1 in PyTorch
        self.elu = nn.ELU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.elu(x)
        x = self.conv2(x)
        x = self.elu(x)
        return x

In [ ]:
convblk = ConvBlock(25,32)
convblk(torch.randn(25,64)).shape

In [12]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.MaxPool1d(2)  # Maintains channel dimension

    def forward(self, x):
        x = self.conv_block(x)
        p = self.pool(x)  # Pooling reduces sequence length by half
        return x, p
    
# encoder = EncoderBlock(25,32)
# o = encoder(torch.randn(25,64))
# o[0].shape,o[1].shape
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.transpose = nn.ConvTranspose1d(in_channels, out_channels, kernel_size=2, stride=2, padding=0)  # Adjust padding for desired output size
        self.conv_block = ConvBlock(in_channels, out_channels)  # Doubled channels after concatenation

    def forward(self, x, skip):
        x = self.transpose(x)  # Doubles sequence length, maintains channels
#         print(x.shape,skip.shape)
        x = torch.cat([x, skip],dim=1)  # Concatenate features, doubles channels
#         print(x.shape)
        x = self.conv_block(x)
        return x



In [ ]:
# dec = DecoderBlock(512,256)
# dec(torch.randn(512,4),torch.randn(256,8)).shape

In [13]:
class UNet(nn.Module):
    def __init__(self, input_shape):
        super(UNet, self).__init__()
        self.zero_pad = nn.ZeroPad1d(2)  # Assuming you meant ZeroPadding2d (for 2D data)

        # Encoder (adjust channel numbers and block definitions as needed)
        self.s00 = EncoderBlock(input_shape[0], 32)  # Input channels, output channels
        self.s0 = EncoderBlock(32, 64)
        self.s1 = EncoderBlock(64, 128)
        self.s2 = EncoderBlock(128, 256)

        # Bottleneck
        self.b1 = ConvBlock(256, 512)  # Maintain channel dimension

        # Decoder (adjust channel numbers and block definitions as needed)
        self.d3 = DecoderBlock(512, 256)  # Input channels, skip channels, output channels
        self.d4 = DecoderBlock(256, 128)
        self.d5 = DecoderBlock(128, 64)
        self.d6 = DecoderBlock(64, 32)

        # Output
        self.outputs = nn.Conv1d(32, 6, kernel_size=1, padding="same")  # Adjust output channels as needed
        self.cropping = nn.ConstantPad1d((2, 0), value=0)  # Assuming you want to remove padding at the end

    def forward(self, x):
        x = self.zero_pad(x)  # Pad input if necessary (assuming 2D data)

        # Encoder
#         print(x.shape)
        s00, p00 = self.s00(x)
#         print(s00.shape,p00.shape)
        s0, p0 = self.s0(p00)
#         print(s0.shape,p0.shape)
        s1, p1 = self.s1(p0)
#         print(s1.shape,p1.shape)
        s2, p2 = self.s2(p1)
#         print(s2.shape,p2.shape)

        # Bottleneck
        b1 = self.b1(p2)
#         print(b1.shape)

        # Decoder
        d3 = self.d3(b1, s2)
#         print(d3.shape)
        d4 = self.d4(d3, s1)
#         print(d4.shape)
        d5 = self.d5(d4, s0)
#         print(d5.shape)
        d6 = self.d6(d5, s00)
#         print(d6.shape)

        # Output
        outputs = self.outputs(d6)
#         print(outputs.shape)
        outputs = outputs[:,:,2:-2]
        
        
#         outputs = self.cropping(outputs)  # Remove padding if necessary
#         print(outputs.permute((0,2,1)).shape)

        return outputs.permute((0,2,1))

In [ ]:
unet = UNet((25,64))

In [ ]:
x = unet(torch.randn(1,25,60))
print(x.shape)
# for i in x:
#     print(x.shape)

In [14]:
class CustomModel_Unet_MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_size):
        super(CustomModel_Unet_MLP, self).__init__()

        self.unet = UNet((25,64))
        self.hidden_size = hidden_size
        self.inp_shape = input_dim
        self.n_layers = 6
        self.activation_fn = 'elu'

       # MLP layers
        self.dense1 = nn.Linear(self.inp_shape, 3 * self.hidden_size)
        self.dense2 = nn.Linear(3 * self.hidden_size, 2 * self.hidden_size)
        self.dropout = nn.Dropout(p=0.2)
        self.batch_norm = nn.BatchNorm1d(2 * self.hidden_size)
        self.skip_connections = nn.ModuleList([])
        for _ in range(self.n_layers):
            self.skip_connections.append(nn.Linear(2 * self.hidden_size, 2 * self.hidden_size))
            if self.activation_fn == 'relu':
                self.skip_connections.append(nn.ReLU())
            elif self.activation_fn == 'elu':
                self.skip_connections.append(nn.ELU())
            elif self.activation_fn == 'leakyRelu':
                self.skip_connections.append(nn.LeakyReLU(negative_slope=0.15))
        self.dense3 = nn.Linear(2 * self.hidden_size, 3 * self.hidden_size)
        self.dense_out = nn.Linear(3 * self.hidden_size, output_dim - (60 * 6))  # Adjust output size

    def forward(self, x):
        x1 = x[:, :360]
        x2 = x[:, 376:]
        x3 = x[:, 360:376]

        x3 = x3.unsqueeze(1).expand(-1, 60, -1)
        x1 = x1.view(-1, 60, 6)
        x2 = x2.view(-1, 60, 3)

        cnn_input = torch.cat([x1, x3, x2], dim=-1)
        new_shape = (cnn_input.shape[0],cnn_input.shape[2],cnn_input.shape[1])
        cnn_out = self.unet(cnn_input.view(new_shape))
#         print(cnn_out.shape)
        
       # MLP part
        x = self.dense1(x)
        x = self.dense2(x)
        skip = x  # Store initial dense layer output for skip connections

        for i in range(self.n_layers):
            x = self.skip_connections[2 * i](x)
            x = self.skip_connections[2 * i + 1](x)
            x = self.dropout(x)
            x = self.batch_norm(x)
            x = x + skip  # Add skip connection
            skip = x  # Update skip connection for next layer

        x = self.dense3(x)
        ann_out = self.dense_out(x)
#         print(ann_out.shape)
        cnn_out = cnn_out.reshape(-1,  cnn_out.size(1) * cnn_out.size(2))
#         print(ann_out.shape,cnn_out.shape)
        # Concatenate outputs
        x = torch.cat((cnn_out, ann_out), dim=1)
        return x

In [ ]:
unet_mlp = CustomModel_Unet_MLP(556,368,556+368)
x = unet_mlp(torch.tensor(x_train))

In [ ]:
model = CustomModel_Unet_MLP(556,368,556+368).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)

print("Time after all preparations:", format_time(time.time()-ts), flush=True)

In [15]:
from sklearn.metrics import r2_score
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()
input_dim = 556  # Example input dimension
output_dim = 368  # Example output dimension
hidden_size = 556+368  # Example hidden size

model = CustomModel_Unet_MLP(input_dim, output_dim, hidden_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.load_state_dict(torch.load('/kaggle/input/files2-leap/best_model_weights (1).pth'))

<All keys matched successfully>

In [ ]:
# Dummy data for illustration


criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
# scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=100, epochs=10)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=SCHEDULER_FACTOR, patience=SCHEDULER_PATIENCE)
# Assuming train_loader and val_loader are defined
num_epochs = 10

    
num_epochs = 80
best_val_loss = float('inf')
best_r2_score = 0.3425
best_model_state = None
patience_count = 0
PATCIENCE = 10
PRINT_FREQ = 50

for epoch in range(num_epochs):
    print("")
    model.train()
    total_loss = 0
    steps = 0
    ts = time.time()

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
#         scheduler.step()

        total_loss += loss.item()
        steps += 1

        if (batch_idx + 1) % PRINT_FREQ == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            elapsed_time = time.time() - ts
            print(f'  Epoch: {epoch+1}',\
                  f'  Batch: {batch_idx + 1}/{len(train_loader)}',\
                  f'  Train Loss: {total_loss / steps:.4f}',\
                  f'  LR: {current_lr:.1e}',\
                  f'  Time: {elapsed_time:.2f}s', flush=True)
            total_loss = 0
            steps = 0

    model.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)

    r2 = 0
    r2_broken = []
    r2_broken_names = []
    for i in range(len(TARGET_COLS)):
        r2_i = r2_score(y_true[:, i].cpu(), all_outputs[:, i].cpu())
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
    r2 /= len(TARGET_COLS)

    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')

    scheduler.step(avg_val_loss)

    if r2 > best_r2_score:
        best_model_state = model.state_dict()
        best_r2_score = r2
        model_weights_path = 'best_model_weights.pth'
        torch.save(model.state_dict(), model_weights_path)
        print("R2 score improved, saving new best model. model saved to best_model_weights.pth")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_count = 0
        print("Validation loss decreased, resetting patience counter.")
    else:
        patience_count += 1
        print(f"No improvement in validation loss for {patience_count} epochs.")
        
    if patience_count >= PATIENCE:
        print("Stopping early due to no improvement in validation loss.")
        break

model.load_state_dict(best_model_state)
print("Training complete. Best R2 score: {:.4f}".format(best_r2_score))


  Epoch: 1   Batch: 50/174   Train Loss: 0.3388   LR: 1.0e-03   Time: 134.86s
  Epoch: 1   Batch: 100/174   Train Loss: 0.3089   LR: 1.0e-03   Time: 268.61s
  Epoch: 1   Batch: 150/174   Train Loss: 0.3084   LR: 1.0e-03   Time: 400.29s

Epoch: 1  Val Loss: 0.3234  R2 score: 0.3094
97 targets were excluded during evaluation of R2 score.
Validation loss decreased, resetting patience counter.

  Epoch: 2   Batch: 50/174   Train Loss: 0.3059   LR: 1.0e-03   Time: 132.05s
  Epoch: 2   Batch: 100/174   Train Loss: 0.3002   LR: 1.0e-03   Time: 264.27s
  Epoch: 2   Batch: 150/174   Train Loss: 0.3031   LR: 1.0e-03   Time: 395.34s

Epoch: 2  Val Loss: 0.3398  R2 score: 0.2956
99 targets were excluded during evaluation of R2 score.
No improvement in validation loss for 1 epochs.

  Epoch: 3   Batch: 50/174   Train Loss: 0.3028   LR: 1.0e-03   Time: 131.58s
  Epoch: 3   Batch: 100/174   Train Loss: 0.2961   LR: 1.0e-03   Time: 263.11s
  Epoch: 3   Batch: 150/174   Train Loss: 0.2985   LR: 1.0e-0

In [ ]:
model.load_state_dict(best_model_state)
model_weights_path = 'best_model_weights.pth'
torch.save(model.state_dict(), model_weights_path)
model_path = 'best_model.pth'
torch.save(model, model_path)

In [ ]:
best_val_loss = float('inf')
best_r2_score = -1000
best_model_state = None
patience_count = 0
r2score = R2Score(num_outputs=len(TARGET_COLS)).to(device)
EPOCHS = 5
for epoch in range(EPOCHS):
    print("")
    model.train()
    total_loss = 0
    steps = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        steps += 1

        if (batch_idx + 1) % PRINT_FREQ == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            elapsed_time = format_time(time.time() - ts)
            print(f'  Epoch: {epoch+1}',\
                  f'  Batch: {batch_idx + 1}/{len(train_loader)}',\
                  f'  Train Loss: {total_loss / steps:.4f}',\
                  f'  LR: {current_lr:.1e}',\
                  f'  Time: {elapsed_time}', flush=True)
            total_loss = 0
            steps = 0
    

    model.eval()
    val_loss = 0
    y_true = torch.tensor([], device=device)
    all_outputs = torch.tensor([], device=device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            y_true = torch.cat((y_true, labels), 0)
            all_outputs = torch.cat((all_outputs, outputs), 0)
    r2=0
    r2_broken = []
    r2_broken_names = []
    for i in range(368):
        r2_i = r2score(all_outputs[:, i], y_true[:, i])
        if r2_i > 1e-6:
            r2 += r2_i
        else:
            r2_broken.append(i)
            r2_broken_names.append(FEAT_COLS[i])
#     r2 = r2score(all_outputs,y_true)
    r2 /= 368

    avg_val_loss = val_loss / len(val_loader)
    print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')
    print(f'{len(r2_broken)} targets were excluded during evaluation of R2 score.')
    # print(r2_broken)
    # print(r2_broken_names, flush=True)
   
    scheduler.step(avg_val_loss)

#     if avg_val_loss < best_val_loss:
    if r2 > best_r2_score:
        best_model_state = model.state_dict()
        best_r2_score = r2
        print("R2 score improved, saving new best model.")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_count = 0
        print("Validation loss decreased, resetting patience counter.")
    else:
        patience_count += 1
        print(f"No improvement in validation loss for {patience_count} epochs.")
        
    if patience_count >= PATIENCE:
        print("Stopping early due to no improvement in validation loss.")
        break

del x_train, y_train
gc.collect()

In [ ]:
x_train.shape

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import KFold
from torchmetrics import R2Score

# Custom imports
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


input_dim = x_train.shape[1]
output_dim = y_train.shape[1]
hidden_size = input_dim + output_dim

kf = KFold(n_splits=20, shuffle=True, random_state=42)
MAX_LR = 1e-1
EPOCHS = 50
BATCH_SIZE = 10000
EARLY_PATIENCE = 5
PRINT_FREQ = 10

best_val_loss = float('inf')
best_r2_score = -1000
best_model_state = None
patience_count = 0
r2score = R2Score(num_outputs=output_dim).to(device)

for fold, (train_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    train_dataset = TensorDataset(torch.tensor(x_train[train_idx], dtype=torch.float32), 
                                  torch.tensor(y_train[train_idx], dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(x_train[val_idx], dtype=torch.float32), 
                                torch.tensor(y_train[val_idx], dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    model = CustomModel_Unet_MLP(input_dim, output_dim, hidden_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=MAX_LR)
    criterion = nn.MSELoss()
    scheduler = OneCycleLR(optimizer, max_lr=MAX_LR, epochs=EPOCHS, steps_per_epoch=len(train_loader))
#     early_stopping = EarlyStopping(patience=EARLY_PATIENCE)

    for epoch in range(EPOCHS):
        print("")
        model.train()
        total_loss = 0
        steps = 0
        ts = time.time()
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            steps += 1

            if (batch_idx + 1) % PRINT_FREQ == 0:
                current_lr = optimizer.param_groups[0]["lr"]
                elapsed_time = format_time(time.time() - ts)
                print(f'Epoch: {epoch+1}  Batch: {batch_idx + 1}/{len(train_loader)}  '
                      f'Train Loss: {total_loss / steps:.4f}  LR: {current_lr:.1e}  Time: {elapsed_time}', flush=True)
                total_loss = 0
                steps = 0

        model.eval()
        val_loss = 0
        y_true = torch.tensor([], device=device)
        all_outputs = torch.tensor([], device=device)
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                y_true = torch.cat((y_true, labels), 0)
                all_outputs = torch.cat((all_outputs, outputs), 0)

        r2 = 0
        for i in range(output_dim):
            r2_i = r2score(all_outputs[:, i], y_true[:, i])
            if r2_i > 1e-6:
                r2 += r2_i
        r2 /= output_dim

        avg_val_loss = val_loss / len(val_loader)
        print(f'\nEpoch: {epoch+1}  Val Loss: {avg_val_loss:.4f}  R2 score: {r2:.4f}')

        scheduler.step()

        if r2 > best_r2_score:
            best_model_state = model.state_dict()
            best_r2_score = r2
            print("R2 score improved, saving new best model.")
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_count = 0
            print("Validation loss decreased, resetting patience counter.")
        else:
            patience_count += 1
            print(f"No improvement in validation loss for {patience_count} epochs.")
        
        if patience_count >= EARLY_PATIENCE:
            print("Stopping early due to no improvement in validation loss.")
            break

    model.load_state_dict(best_model_state)
    break  # Remove this line to run for all folds


# Submission

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

model.load_state_dict(best_model_state)
model.eval()


df_test = pl.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/test.csv")

for col in FEAT_COLS:
    df_test = df_test.with_columns(pl.col(col).cast(pl.Float32))
df_test,list_vector_features = normalize(df_test,FEAT_COLS)
x_test = df_test.select(FEAT_COLS).to_numpy()
# x_test = x_test.drop(low_variance_cols).to_numpy()

# x_test = (x_test - mx.reshape(1,-1)) / sx.reshape(1,-1)
output_size = 368
predt = np.zeros([x_test.shape[0], output_size], dtype=np.float32)

i1 = 0
for i in range(10000):
    i2 = np.minimum(i1 + BATCH_SIZE, x_test.shape[0])
    if i1 == i2:  # Break the loop if range does not change
        break

    # Convert the current slice of xt to a PyTorch tensor
    inputs = torch.from_numpy(x_test[i1:i2, :]).float().to(device)

    # No need to track gradients for inference
    with torch.no_grad():
        outputs = model(inputs)  # Get model predictions
        predt[i1:i2, :] = outputs.cpu().numpy()  # Store predictions in predt

    i1 = i2  # Update i1 to the end of the current batch

    if i2 >= x_test.shape[0]:
        break

for i in range(sy.shape[0]):
    if sy[i] < MIN_STD * 1.1:
        predt[:,i] = 0

predt = predt * sy.reshape(1,-1) + my.reshape(1,-1)

ss = pd.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/sample_submission.csv")
ss.iloc[:,1:] = predt

del predt
gc.collect()

use_cols = []
for i in range(27):
    use_cols.append(f"ptend_q0002_{i}")

ss2 = pd.read_csv(DATA_PATH + "leap-atmospheric-physics-ai-climsim/sample_submission.csv")
df_test = df_test.to_pandas()
for col in use_cols:
    ss[col] = -df_test[col.replace("ptend", "state")]*ss2[col]/1200.

test_polars = pl.from_pandas(ss[["sample_id"]+TARGET_COLS])
test_polars.write_csv("submission.csv")

print("Total time:", format_time(time.time()-ts))

In [ ]:
!pip install kaggle -q

In [ ]:
import os
os.environ["KAGGLE_USERNAME"]="sanket086sanket"
os.environ["KAGGLE_KEY"]="dae4c6b7590cd576181b55c7356952b0"

In [ ]:
!kaggle competitions submit -c leap-atmospheric-physics-ai-climsim -f submission.csv -m "nn_2"